# Experimentando con Rankings Deportivos
## Implementación en C++ 💪💪
Let's get this show on the road


Definir los path al ejecutable de python 3.6 y sus librerías,
de acuerdo al virtual env que estén corriendo.

In [1]:
!rm -rf ./build
!mkdir ./build
!cd ./build && git submodule init
!cd ./build && git submodule update
!cp -r ../src ./build/src
!rm -rf ./build/src/CMakeCache.txt
!cd ./build/src/ && cmake \
  -DCMAKE_BUILD_TYPE=Release -DCMAKE_CXX_FLAGS="-Wno-sign-compare" .
!cd ./build/src && make
!cp ./build/src/tp1 ./src

-- The C compiler identification is GNU 9.3.0
-- The CXX compiler identification is GNU 9.3.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Configuring done
-- Generating done
-- Build files have been written to: /home/roni/Documents/metnum/metnum-tp1/entregable/notebooks/build/src
Scanning dependencies of target tp1
[ 16%] Building CXX object CMakeFiles/tp1.dir/Clases/Equipo.cpp.o
[ 33%] Linking CXX executable tp1
[100%] Built target tp1


In [2]:
from commons import *

### NBA
Vamos a realizar una prueba con el dataset de la nba para cada metodo y ver como las posiciones de cada equipo varia para cada metodo

In [ ]:
df_nba = calculateRatings("../data/nba_2016_scores.dat","../data/nba_2016_teams.csv")
plotAllMethods(df_nba)

### Liga los amigos
Un grupo de 10 amigos se junta a jugar una liga de Winning Eleven 2006 todos los sabados. Cada jugador se enfrenta a otro, pero no siempre todos los jugadores pueden jugar asi que no todos juegan la misma cantidad de partidos. Por lo tanto, teniendo en cuenta de esta diferencia, los amigos quieren saber cual es la mejor forma de calcular un ranking para saber las posiciones en su liga. Ademas Jorge, uno de los amigos, dice que el es mejor que Juan porque juega contra amigos que jugan mejor, y quiere que esto quede reflejado en su tabla asi juan no sale primero por jugar siempre contra los peores.

In [ ]:
df_amigos1 = calculateRatings("../data/liga_los_amigos_scores1.dat","../data/liga_los_amigos_teams.csv")
plotAllMethods(df_amigos1)

In [ ]:
df_amigos1.loc[df_amigos1["Equipo"].isin(["Juan","Jorge"])].reset_index(drop=True)

Luego de hacer los calculos para la primera fecha Juan, como Jorge habia adelantado, se enfrento contra los peores (jugadores que perdieron todos los partidos que jugaron) y les gano por goleada, a ver los resultados de los 3 rankings (CMM, WP y BP), CMM fue el unico que pudo cumplir las expectativas de Jorge, que le dio mas puntuacion ya que jugo contra jugadores que ganaron partidos. En WP salieron empadatados porque ambos ganaron todos sus partidos, y en BP Juan supero a Jorge ya que el goleo a sus contrincantes.

Al siguiente sabado se volvieron a juntar los amigos se volvieron a juntar, pero esta vez Jorge y Juan no pudieron estar presentes. Ambos con una gran ventaja sobre sus competidores piensan que sus posiciones no van a ser modificadas a menos que algun competidor los supere con amplias victorias. En la segunda fecha los competidores que habian perdido contra Juan, los peores, en su mayoria ganaron sus enfrentamientos. Entonces al ver la tabla de posiciones segun CMM Jorge ya no era el primero, ya que Juan lo supero; pero ninguno de los dos habia jugado. Esto paso debido a que los ratings de los jugadores que le gano Juan subieron y en CMM ganarle a jugadores con buen rating te da mas rating. 

In [ ]:
df_amigos2 = calculateRatings("../data/liga_los_amigos_scores2.dat","../data/liga_los_amigos_teams.csv")
plotAllMethods(df_amigos2)

In [ ]:
df_amigos2.loc[df_amigos2["Equipo"].isin(["Juan","Jorge"])].reset_index(drop=True)

In [ ]:
df_amigos_diff = pd.DataFrame(df_amigos2["id"]).copy()
df_amigos_diff["Equipo"] = df_amigos2["Equipo"]
df_amigos_diff["diff CMM"] = df_amigos2["CMM"].astype(float) - df_amigos1["CMM"].astype(float)
df_amigos_diff["diff WP"] = df_amigos2["WP"].astype(float) - df_amigos1["WP"].astype(float)
df_amigos_diff["diff BP"] = df_amigos2["BP"].astype(float) - df_amigos1["BP"].astype(float)

df_amigos_diff.loc[df_amigos_diff["Equipo"].isin(["Juan","Jorge"])].reset_index(drop=True)

In [ ]:
cmm_amigos1_rank = df_amigos1.sort_values('CMM',ascending=True)["id"]
cmm_amigos2_rank = df_amigos2.sort_values('CMM', ascending=True)["id"]
teams = df_amigos2["Equipo"]

positions = range(1,len(teams)+1)

w=0.35
cmm_amigos1_pos = [t - w/2 for t in cmm_amigos1_rank]
cmm_amigos2_pos = [t + w/2 for t in cmm_amigos2_rank]

fig, ax = plt.subplots()

plt.bar(cmm_amigos1_pos,positions,label="Fecha 1", width=w)
plt.bar(cmm_amigos2_pos, positions, label="Fecha 2", width=w)

plt.ylabel("Posicion", fontsize=18)
plt.xlabel("Equipos", fontsize=18)
plt.xticks(range(1,len(teams)+1), teams, rotation=75, fontsize=12)
plt.yticks(positions, range(len(positions),0,-1))
fig.set_size_inches(20, 8)
plt.legend(fontsize=14)
plt.title("Evolución de posiciones en base al ranking CMM", fontsize=22)
ax.yaxis.grid(color='dimgrey')
plt.show()